In [61]:
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import seaborn as sns
import numpy as np
pd.set_option('display.max_columns', 700)
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore") 

In [62]:
df_train = pd.read_csv('TRAIN (1).csv')
df_train['order_date'] = pd.to_datetime(df_train['order_date'], format = '%m/%d/%y')
df_train['evsd'] = pd.to_datetime(df_train['evsd'], format = '%m/%d/%y')
df_train['tag'] = 'train'

print(df_train.shape)
df_train.head()

(259287, 14)


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag
0,1,JFK2,540,30718,67271,2022-11-02,2022-11-14,AFT Lite,54,54,54,4,9,train
1,1,JFK2,540,30718,125389,2022-11-20,2022-12-01,AFT Lite,60,60,60,5,3,train
2,1,JFK2,648,16750,11165,2022-09-30,2022-10-03,AFT Lite,6,6,6,1,2,train
3,1,JFK2,648,16750,27008,2022-10-14,2022-10-17,AFT Lite,6,6,6,2,2,train
4,1,JFK2,7,8748,33598,2022-11-15,2022-11-16,AFT Lite,8,8,8,1,1,train


In [63]:
# Create a new column that combines the values of the existing columns
df_train['unique_id'] = df_train.apply(lambda row: f"{row['warehouse_id']}_{row['d_id']}_{row['isbn_id']}_{row['order_id']}", axis=1)

# Drop duplicates based on the new column
df_unique = df_train.drop_duplicates(subset=['unique_id'])

df_unique.shape, df_train.shape

((101777, 15), (259287, 15))

In [64]:
df_train.head()

,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id
0,1,JFK2,540,30718,67271,2022-11-02,2022-11-14,AFT Lite,54,54,54,4,9,train,JFK2_540_30718_67271
1,1,JFK2,540,30718,125389,2022-11-20,2022-12-01,AFT Lite,60,60,60,5,3,train,JFK2_540_30718_125389
2,1,JFK2,648,16750,11165,2022-09-30,2022-10-03,AFT Lite,6,6,6,1,2,train,JFK2_648_16750_11165
3,1,JFK2,648,16750,27008,2022-10-14,2022-10-17,AFT Lite,6,6,6,2,2,train,JFK2_648_16750_27008
4,1,JFK2,7,8748,33598,2022-11-15,2022-11-16,AFT Lite,8,8,8,1,1,train,JFK2_7_8748_33598


In [65]:
df_train['visibility'].dtype

dtype('int64')

In [66]:
# How long does order usually take to deliver?
# df_unique['order_delivery_days'] = (df_unique['evsd'] - df_unique['order_date']).dt.days
# df_test['order_delivery_days'] = (df_test['evsd'] - df_test['order_date']).dt.days

df_unique['consider'] =   np.where((df_unique['evsd'] - df_unique['order_date']).dt.days  == df_unique['visibility'], True, False)




In [67]:
df_unique['consider'].value_counts(dropna= False)

consider
True     54085
False    47692
Name: count, dtype: int64

In [68]:
df_test = pd.read_csv('Test.csv', encoding = 'latin1')
# df_test.columns = ['LOCATION', 'd_id', 'isbn_id', 'order_id_1', 'order_date', 'evsd',
#        'row_group_ind', 'visibility', 'quantity_ordered', 'quantity_submitted',
#        'quantity_received']
df_test['order_date'] = pd.to_datetime(df_test['order_date'], format = '%m/%d/%y')
df_test['evsd'] = pd.to_datetime(df_test['evsd'], format = '%m/%d/%y')
df_test['tag'] = 'test'
print(df_test.shape)
df_test.head()

(98309, 14)


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag
0,1,JFK2,684,674,45573,2022-12-15,2022-12-19,AFT Lite,42,42,42,3,3,test
1,1,JFK2,1,19822,110331,2022-12-27,2022-12-30,AFT Lite,6,6,6,0,3,test
2,1,JFK2,509,19749,119943,2022-12-15,2022-12-17,AFT Lite,12,12,12,3,1,test
3,1,JFK2,584,19747,43091,2022-12-20,2022-12-23,AFT Lite,12,12,12,4,3,test
4,1,JFK2,540,20260,117854,2022-12-26,2023-01-04,AFT Lite,60,60,60,1,8,test


In [69]:
# Apply same filter on Test Data too
# Create a new column that combines the values of the existing columns
df_test['unique_id'] = df_test.apply(lambda row: f"{row['warehouse_id']}_{row['d_id']}_{row['isbn_id']}_{row['order_id']}", axis=1)
print(df_test.shape)
# Drop duplicates based on the new column
df_test = df_test.drop_duplicates(subset=['unique_id'])

df_test.shape

(98309, 15)


(40996, 15)

In [70]:
df_test['consider'] =   np.where((df_test['evsd'] - df_test['order_date']).dt.days  == df_test['visibility'], True, False)
df_test['consider'].value_counts(dropna= False)

consider
True     22406
False    18590
Name: count, dtype: int64

## Final DATA : ``df_unique`` and ``df_test``

In [71]:
df_unique = df_unique.loc[df_unique['consider'] == True].reset_index(drop=True)
df_test = df_test.loc[df_test['consider'] == True].reset_index(drop=True)


In [46]:
df_unique.columns

Index(['region_id', 'warehouse_id', 'd_id', 'isbn_id', 'order_id',
       'order_date', 'evsd', 'po_source_ind', 'quantity_ordered',
       'quantity_submitted', 'quantity_received', 'row_group_ind',
       'visibility', 'tag', 'unique_id', 'consider'],
      dtype='object')

In [72]:
# Feature Engineering
for col in ['d_id', 'isbn_id', 'order_id']:
  df_unique[col] = df_unique[col].astype(int).astype(str)
  df_test[col] = df_test[col].astype(int).astype(str)
  # df_test_n[col] = df_test_n[col].astype(int).astype(str)

print(df_unique.shape)
display(df_unique.head(2))

print(df_test.shape)
df_test.head(2)

(54085, 16)


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id,consider
0,1,JFK2,7,8748,33598,2022-11-15,2022-11-16,AFT Lite,8,8,8,1,1,train,JFK2_7_8748_33598,True
1,1,JFK2,7,8748,142228,2022-11-01,2022-11-02,AFT Lite,16,16,16,3,1,train,JFK2_7_8748_142228,True


(22406, 16)


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id,consider
0,1,JFK2,1,19822,110331,2022-12-27,2022-12-30,AFT Lite,6,6,6,0,3,test,JFK2_1_19822_110331,True
1,1,JFK2,584,19747,43091,2022-12-20,2022-12-23,AFT Lite,12,12,12,4,3,test,JFK2_584_19747_43091,True


In [73]:

df_unique['order_day'] = df_unique['order_date'].dt.day
df_unique['order_dayofweek'] = df_unique['order_date'].dt.dayofweek
df_unique['order_dayofyear'] = df_unique['order_date'].dt.dayofyear
df_unique['order_month'] = df_unique['order_date'].dt.month

df_unique['delivered_day'] = df_unique['evsd'].dt.day
df_unique['delivered_dayofweek'] = df_unique['evsd'].dt.dayofweek
df_unique['delivered_dayofyear'] = df_unique['evsd'].dt.dayofyear
df_unique['delivered_month'] = df_unique['evsd'].dt.month

df_unique['order_month_sin'] = np.sin(2 * np.pi * df_unique['order_month'] / 12)
df_unique['order_month_cos'] = np.cos(2 * np.pi * df_unique['order_month'] / 12)

df_unique['order_dayofweek_sin'] = np.sin(2 * np.pi * df_unique['order_dayofweek'] / 7)
df_unique['order_dayofweek_cos'] = np.cos(2 * np.pi * df_unique['order_dayofweek'] / 7)

df_unique['delivered_month_sin'] = np.sin(2 * np.pi * df_unique['delivered_month'] / 12)
df_unique['delivered_month_cos'] = np.cos(2 * np.pi * df_unique['delivered_month'] / 12)

df_unique['delivered_dayofweek_sin'] = np.sin(2 * np.pi * df_unique['delivered_dayofweek'] / 7)
df_unique['delivered_dayofweek_cos'] = np.cos(2 * np.pi * df_unique['delivered_dayofweek'] / 7)

df_unique.drop(['order_month','delivered_month'], axis =1, inplace = True)
display(df_unique.head())


df_test['order_day'] = df_test['order_date'].dt.day
df_test['order_dayofweek'] = df_test['order_date'].dt.dayofweek
df_test['order_dayofyear'] = df_test['order_date'].dt.dayofyear
df_test['order_month'] = df_test['order_date'].dt.month

df_test['delivered_day'] = df_test['evsd'].dt.day
df_test['delivered_dayofweek'] = df_test['evsd'].dt.dayofweek
df_test['delivered_dayofyear'] = df_test['evsd'].dt.dayofyear
df_test['delivered_month'] = df_test['evsd'].dt.month

df_test['order_month_sin'] = np.sin(2 * np.pi * df_test['order_month'] / 12)
df_test['order_month_cos'] = np.cos(2 * np.pi * df_test['order_month'] / 12)

df_test['order_dayofweek_sin'] = np.sin(2 * np.pi * df_test['order_dayofweek'] / 7)
df_test['order_dayofweek_cos'] = np.cos(2 * np.pi * df_test['order_dayofweek'] / 7)

df_test['delivered_month_sin'] = np.sin(2 * np.pi * df_test['delivered_month'] / 12)
df_test['delivered_month_cos'] = np.cos(2 * np.pi * df_test['delivered_month'] / 12)

df_test['delivered_dayofweek_sin'] = np.sin(2 * np.pi * df_test['delivered_dayofweek'] / 7)
df_test['delivered_dayofweek_cos'] = np.cos(2 * np.pi * df_test['delivered_dayofweek'] / 7)

df_test.drop(['order_month','delivered_month'], axis =1, inplace = True)
display(df_test.head())


# df_test_n['order_day'] = df_test_n['order_date'].dt.day
# df_test_n['order_dayofweek'] = df_test_n['order_date'].dt.dayofweek
# df_test_n['order_dayofyear'] = df_test_n['order_date'].dt.dayofyear
# df_test_n['order_month'] = df_test_n['order_date'].dt.month

# df_test_n['delivered_day'] = df_test_n['evsd'].dt.day
# df_test_n['delivered_dayofweek'] = df_test_n['evsd'].dt.dayofweek
# df_test_n['delivered_dayofyear'] = df_test_n['evsd'].dt.dayofyear
# df_test_n['delivered_month'] = df_test_n['evsd'].dt.month

# df_test_n['order_month_sin'] = np.sin(2 * np.pi * df_test_n['order_month'] / 12)
# df_test_n['order_month_cos'] = np.cos(2 * np.pi * df_test_n['order_month'] / 12)

# df_test_n['order_dayofweek_sin'] = np.sin(2 * np.pi * df_test_n['order_dayofweek'] / 7)
# df_test_n['order_dayofweek_cos'] = np.cos(2 * np.pi * df_test_n['order_dayofweek'] / 7)

# df_test_n['delivered_month_sin'] = np.sin(2 * np.pi * df_test_n['delivered_month'] / 12)
# df_test_n['delivered_month_cos'] = np.cos(2 * np.pi * df_test_n['delivered_month'] / 12)

# df_test_n['delivered_dayofweek_sin'] = np.sin(2 * np.pi * df_test_n['delivered_dayofweek'] / 7)
# df_test_n['delivered_dayofweek_cos'] = np.cos(2 * np.pi * df_test_n['delivered_dayofweek'] / 7)

# df_test_n.drop(['order_month','delivered_month'], axis =1, inplace = True)
# display(df_test_n.head())


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id,consider,order_day,order_dayofweek,order_dayofyear,delivered_day,delivered_dayofweek,delivered_dayofyear,order_month_sin,order_month_cos,order_dayofweek_sin,order_dayofweek_cos,delivered_month_sin,delivered_month_cos,delivered_dayofweek_sin,delivered_dayofweek_cos
0,1,JFK2,7,8748,33598,2022-11-15,2022-11-16,AFT Lite,8,8,8,1,1,train,JFK2_7_8748_33598,True,15,1,319,16,2,320,-0.500000,0.866025,0.781831,0.623490,-0.500000,0.866025,0.974928,-0.222521
1,1,JFK2,7,8748,142228,2022-11-01,2022-11-02,AFT Lite,16,16,16,3,1,train,JFK2_7_8748_142228,True,1,1,305,2,2,306,-0.500000,0.866025,0.781831,0.623490,-0.500000,0.866025,0.974928,-0.222521
2,1,JFK2,7,8748,27330,2022-11-05,2022-11-06,AFT Lite,0,24,0,6,1,train,JFK2_7_8748_27330,True,5,5,309,6,6,310,-0.500000,0.866025,-0.974928,-0.222521,-0.500000,0.866025,-0.781831,0.623490
3,1,JFK2,7,17780,122956,2022-11-12,2022-11-13,AFT Lite,36,36,36,3,1,train,JFK2_7_17780_122956,True,12,5,316,13,6,317,-0.500000,0.866025,-0.974928,-0.222521,-0.500000,0.866025,-0.781831,0.623490
4,1,JFK2,7,27826,175768,2022-10-11,2022-10-12,AFT Lite,24,24,24,5,1,train,JFK2_7_27826_175768,True,11,1,284,12,2,285,-0.866025,0.500000,0.781831,0.623490,-0.866025,0.500000,0.974928,-0.222521


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id,consider,order_day,order_dayofweek,order_dayofyear,delivered_day,delivered_dayofweek,delivered_dayofyear,order_month_sin,order_month_cos,order_dayofweek_sin,order_dayofweek_cos,delivered_month_sin,delivered_month_cos,delivered_dayofweek_sin,delivered_dayofweek_cos
0,1,JFK2,1,19822,110331,2022-12-27,2022-12-30,AFT Lite,6,6,6,0,3,test,JFK2_1_19822_110331,True,27,1,361,30,4,364,-2.449294e-16,1.0,0.781831,0.623490,-2.449294e-16,1.0,-0.433884,-0.900969
1,1,JFK2,584,19747,43091,2022-12-20,2022-12-23,AFT Lite,12,12,12,4,3,test,JFK2_584_19747_43091,True,20,1,354,23,4,357,-2.449294e-16,1.0,0.781831,0.623490,-2.449294e-16,1.0,-0.433884,-0.900969
2,1,JFK2,540,32203,112020,2022-12-07,2022-12-16,AFT Lite,18,18,18,5,9,test,JFK2_540_32203_112020,True,7,2,341,16,4,350,-2.449294e-16,1.0,0.974928,-0.222521,-2.449294e-16,1.0,-0.433884,-0.900969
3,1,JFK2,509,10609,26126,2022-12-08,2022-12-10,AFT Lite,18,18,18,6,2,test,JFK2_509_10609_26126,True,8,3,342,10,5,344,-2.449294e-16,1.0,0.433884,-0.900969,-2.449294e-16,1.0,-0.974928,-0.222521
4,1,JFK2,509,8427,26144,2022-12-29,2022-12-31,AFT Lite,0,36,0,1,2,test,JFK2_509_8427_26144,True,29,3,363,31,5,365,-2.449294e-16,1.0,0.433884,-0.900969,-2.449294e-16,1.0,-0.974928,-0.222521


In [74]:
df = pd.concat([df_unique, df_test]).reset_index(drop = True)

df_grouped = df[['d_id', 'evsd']].groupby('d_id').nunique()

# Filter d_id with more than 10 time periods
ids = list(df_grouped[df_grouped['evsd']>=10].index)
df = df[df['d_id'].isin(ids)].reset_index(drop = True).copy()



In [75]:

vc_cols = ['delivered_day', 'delivered_dayofweek','order_day', 'order_dayofweek']
for col in vc_cols:
    temp_dict = df_unique[col].value_counts(normalize=True).to_dict()
    df[f"{col}_VC"] = df[col].map(temp_dict)

In [ ]:
df = pd.get_dummies(df, columns=['row_group_ind'])

In [82]:
req_cols = [col for col in df.columns if col not in ['region_id', 'warehouse_id', 'd_id', 'isbn_id',
                                                            'order_month_cos', 'order_month_sin','order_id',
                                                              'order_date', 'evsd', 'po_source_ind', 'tag',
                                                                'unique_id', 'row_group_ind','quantity_received', 
                                                                'consider', 'order_dayofyear', 'delivered_dayofyear',
                                                                'delivered_day', 'delivered_dayofweek','order_day', 'order_dayofweek', 
                                                                'quantity_received_scaled', 
                                                                'order_dayofweek_sin', 'order_dayofweek_cos', 
                                                                'delivered_month_sin', 'delivered_month_cos', 
 'delivered_dayofweek_sin', 'delivered_dayofweek_cos']]

In [83]:
def mae(y_true, predictions):
    y_true, predictions = np.array(y_true), np.array(predictions)
    return np.mean(np.abs(y_true - predictions))

def new_metric(df):
    df['mae'] = df[['quantity_received', 'preds']].apply(lambda row: mae(row['quantity_received'], row['preds']), axis=1)
    final_error = df['mae'].sum()/ df['preds'].sum()
    return final_error

In [84]:
req_cols

['quantity_ordered',
 'quantity_submitted',
 'visibility',
 'delivered_day_VC',
 'delivered_dayofweek_VC',
 'order_day_VC',
 'order_dayofweek_VC',
 'row_group_ind_0',
 'row_group_ind_1',
 'row_group_ind_2',
 'row_group_ind_3',
 'row_group_ind_4',
 'row_group_ind_5',
 'row_group_ind_6',
 'row_group_ind_7']

In [89]:
from sklearn import preprocessing

In [90]:
used_cols = ['quantity_ordered',
 'quantity_submitted',
 'visibility',
 'row_group_ind_3',
 'row_group_ind_4',
 'row_group_ind_2',
 'row_group_ind_5',
 'row_group_ind_1',
 'row_group_ind_6',
 'row_group_ind_7',
 'row_group_ind_0',

 'delivered_day_VC',
 'delivered_dayofweek_VC',
 'order_day_VC',
 'order_dayofweek_VC']

In [94]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import r2_score, mean_absolute_error

# Define min-max scaler
def minmax_scaler(data, min_val=None, max_val=None):
    if min_val is None:
        min_val = np.min(data)
    if max_val is None:
        max_val = np.max(data)
    scaled = (data - min_val) / (max_val - min_val)
    return scaled, min_val, max_val

# Define inverse min-max scaler
def inverse_minmax_scaler(data_scaled, min_val, max_val):
    return data_scaled * (max_val - min_val) + min_val

# Load data
# X_train, X_test, y_train, y_test = load_data()
X = df[used_cols].copy()
y = df[['quantity_received']].copy()
X_train, X_test = train_test_split(X, test_size=0.3, random_state=42)
y_train, y_test = train_test_split(y, test_size=0.3, random_state=42)


# Quantile Transform
scaler_s = preprocessing.QuantileTransformer()
X_train[['quantity_ordered', 'quantity_submitted', 'visibility']] = \
    scaler_s.fit_transform(X_train[['quantity_ordered', 'quantity_submitted', 'visibility']])

X_test[['quantity_ordered', 'quantity_submitted', 'visibility']] = \
    scaler_s.transform(X_test[['quantity_ordered', 'quantity_submitted', 'visibility']])

# Scale target variable
y_train_scaled, y_min, y_max = minmax_scaler(y_train)
y_test_scaled, _, _ = minmax_scaler(y_test, y_min, y_max)

from catboost import CatBoostRegressor

# Initialize the CatBoost Regressor with the desired parameters
model = CatBoostRegressor(iterations=2000,
                          learning_rate=0.0001,
                          depth=10,
                          l2_leaf_reg=5,
                          random_seed=42,
                          loss_function='MAE',
                          od_type='Iter',
                          od_wait=20,
                          bagging_temperature=0.8,
                          verbose=500)

# Fit the model on the training data
# model.fit(X_train, y_train,
#           eval_set=(X_test, y_test),
#           early_stopping_rounds=100,
#           use_best_model=True)
model.fit(X_train, y_train_scaled,
          eval_set=(X_test, y_test_scaled),
          early_stopping_rounds=300,
          use_best_model=True)

# Evaluate model on test set
y_pred_scaled = model.predict(X_test)
# y_pred = model.predict(X_test)
y_pred = inverse_minmax_scaler(y_pred_scaled, y_min, y_max)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('R2:', r2)
print('MAE:', mae)


0:	learn: 0.0079798	test: 0.0082866	best: 0.0082866 (0)	total: 56.8ms	remaining: 1m 53s
500:	learn: 0.0077738	test: 0.0080768	best: 0.0080768 (500)	total: 16.6s	remaining: 49.8s
1000:	learn: 0.0075770	test: 0.0078766	best: 0.0078766 (1000)	total: 32.5s	remaining: 32.4s
1500:	learn: 0.0073911	test: 0.0076877	best: 0.0076877 (1500)	total: 48.4s	remaining: 16.1s
1999:	learn: 0.0072123	test: 0.0075058	best: 0.0075058 (1999)	total: 1m 4s	remaining: 0us

bestTest = 0.007505817054
bestIteration = 1999

R2: 0.032213700926146505
MAE: 11.260206055022508


In [95]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import r2_score, mean_absolute_error

# Define min-max scaler
def minmax_scaler(data, min_val=None, max_val=None):
    if min_val is None:
        min_val = np.min(data)
    if max_val is None:
        max_val = np.max(data)
    scaled = (data - min_val) / (max_val - min_val)
    return scaled, min_val, max_val

# Define inverse min-max scaler
def inverse_minmax_scaler(data_scaled, min_val, max_val):
    return data_scaled * (max_val - min_val) + min_val

# Load data
# X_train, X_test, y_train, y_test = load_data()
X = df[used_cols].copy()
y = df[['quantity_received']].copy()
X_train, X_test = train_test_split(X, test_size=0.3, random_state=42, shuffle=True)
y_train, y_test = train_test_split(y, test_size=0.3, random_state=42, shuffle=True)


# Quantile Transform
scaler_s = preprocessing.QuantileTransformer()
X_train[['quantity_ordered', 'quantity_submitted', 'visibility']] = \
    scaler_s.fit_transform(X_train[['quantity_ordered', 'quantity_submitted', 'visibility']])

X_test[['quantity_ordered', 'quantity_submitted', 'visibility']] = \
    scaler_s.transform(X_test[['quantity_ordered', 'quantity_submitted', 'visibility']])




# Scale target variable
y_train_scaled, y_min, y_max = minmax_scaler(y_train)
y_test_scaled, _, _ = minmax_scaler(y_test, y_min, y_max)


# Convert the NumPy array to a TensorFlow tensor
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)


# Define model
# model = tf.keras.Sequential([
#   tf.keras.layers.Dense(128, activation='relu', input_shape=(len(used_cols),)),
#   tf.keras.layers.Dense(64, activation='relu'),
#   tf.keras.layers.Dense(32, activation='relu'),
#   tf.keras.layers.Dense(1)
# ])

from tensorflow.keras.layers import LeakyReLU

model = tf.keras.Sequential([
  tf.keras.layers.Dense(128, input_shape=(len(used_cols),)),
  LeakyReLU(alpha=0.1),
  tf.keras.layers.Dense(64),
  LeakyReLU(alpha=0.1),
  tf.keras.layers.Dense(32),
  LeakyReLU(alpha=0.1),
  tf.keras.layers.Dense(1)
])

# Compile model
optimizer = tf.keras.optimizers.Adam(0.0001)

model.compile(optimizer=optimizer,
              loss='mae',
              metrics=['mae', 'mse'])

# Train model
history = model.fit(X_train, y_train_scaled, epochs=10, batch_size=12,
                    validation_split=0.2)

# Evaluate model on test set
y_pred_scaled = model.predict(X_test)
y_pred = inverse_minmax_scaler(y_pred_scaled, y_min, y_max)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('R2:', r2)
print('MAE:', mae)


Epoch 1/10
3568/3568 [==============================] - 17s 4ms/step - loss: 0.0064 - mae: 0.0064 - mse: 2.7672e-04 - val_loss: 0.0048 - val_mae: 0.0048 - val_mse: 2.5248e-04
Epoch 2/10
3568/3568 [==============================] - 12s 3ms/step - loss: 0.0051 - mae: 0.0051 - mse: 2.2077e-04 - val_loss: 0.0049 - val_mae: 0.0049 - val_mse: 2.4364e-04
Epoch 3/10
3568/3568 [==============================] - 11s 3ms/step - loss: 0.0048 - mae: 0.0048 - mse: 2.1296e-04 - val_loss: 0.0044 - val_mae: 0.0044 - val_mse: 2.4751e-04
Epoch 4/10
3568/3568 [==============================] - 16s 5ms/step - loss: 0.0047 - mae: 0.0047 - mse: 2.0843e-04 - val_loss: 0.0044 - val_mae: 0.0044 - val_mse: 2.3435e-04
Epoch 5/10
3568/3568 [==============================] - 13s 4ms/step - loss: 0.0045 - mae: 0.0045 - mse: 2.0272e-04 - val_loss: 0.0045 - val_mae: 0.0045 - val_mse: 2.3423e-04
Epoch 6/10
3568/3568 [==============================] - 10s 3ms/step - loss: 0.0044 - mae: 0.0044 - mse: 2.0053e-04 - val_los

In [ ]:
def mae(y_true, predictions):
    y_true, predictions = np.array(y_true), np.array(predictions)
    return np.mean(np.abs(y_true - predictions))

def new_metric(df):
    df['mae'] = df[['quantity_received', 'preds']].apply(lambda row: mae(row['quantity_received'], row['preds']), axis=1)
    final_error = df['mae'].sum()/ df['preds'].sum()
    return final_error

In [96]:
import optuna
import numpy as np
import tensorflow as tf
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Define min-max scaler
def minmax_scaler(data, min_val=None, max_val=None):
    if min_val is None:
        min_val = np.min(data)
    if max_val is None:
        max_val = np.max(data)
    scaled = (data - min_val) / (max_val - min_val)
    return scaled, min_val, max_val

# Define inverse min-max scaler
def inverse_minmax_scaler(data_scaled, min_val, max_val):
    return data_scaled * (max_val - min_val) + min_val

# Load data
# X_train, X_test, y_train, y_test = load_data()
X = df[used_cols].copy()
y = df[['quantity_received']].copy()
X_train, X_test = train_test_split(X, test_size=0.3, random_state=42, shuffle=True)
y_train, y_test = train_test_split(y, test_size=0.3, random_state=42, shuffle=True)

# Quantile Transform
scaler_s = preprocessing.QuantileTransformer()
X_train[['quantity_ordered', 'quantity_submitted', 'visibility']] = \
    scaler_s.fit_transform(X_train[['quantity_ordered', 'quantity_submitted', 'visibility']])

X_test[['quantity_ordered', 'quantity_submitted', 'visibility']] = \
    scaler_s.transform(X_test[['quantity_ordered', 'quantity_submitted', 'visibility']])

# Scale target variable
y_train_scaled, y_min, y_max = minmax_scaler(y_train)
y_test_scaled, _, _ = minmax_scaler(y_test, y_min, y_max)

# Convert the NumPy array to a TensorFlow tensor
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)

# Define model function
def create_model(trial):
    # Define number of layers
    n_layers = trial.suggest_int("n_layers", 1, 4)
    # Define number of neurons in each layer
    n_neurons = [trial.suggest_int(f"n_neurons_{i}", 16, 256) for i in range(n_layers)]
    # Define activation function
    activation = trial.suggest_categorical("activation", ["relu", "leaky_relu"])
    # Define learning rate
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)

    # Create model
    model = tf.keras.Sequential()
    for i in range(n_layers):
        if activation == "leaky_relu":
            alpha = trial.suggest_uniform("alpha", 0, 1)
            model.add(tf.keras.layers.Dense(n_neurons[i], input_shape=(len(used_cols),), activation=None))
            model.add(tf.keras.layers.LeakyReLU(alpha=alpha))
        else:
            model.add(tf.keras.layers.Dense(n_neurons[i], activation=activation, input_shape=(len(used_cols),)))
    model.add(tf.keras.layers.Dense(1))

    # Compile model
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss="mae", metrics=["mae", "mse"])

    return model

# Define objective function
def objective(trial):
    model = create_model(trial)

    # Train model
    history = model.fit(X_train, y_train_scaled, epochs=10, batch_size=12, validation_split=0.2)

    # Evaluate model on test set
    y_pred_scaled = model.predict(X_test)
    y_pred = inverse_minmax_scaler(y_pred_scaled, y_min, y_max)
    metric_value = new_metric(pd.concat([y_test, pd.DataFrame(y_pred, columns=["preds"])], axis=1))

    return metric_value


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)


[I 2023-05-03 07:58:48,418] A new study created in memory with name: no-name-a857e804-5f89-43ca-b232-1c7962368f9c


Epoch 1/10
3568/3568 [==============================] - 14s 4ms/step - loss: 0.0112 - mae: 0.0112 - mse: 9.7706e-04 - val_loss: 0.0065 - val_mae: 0.0065 - val_mse: 3.2477e-04
Epoch 2/10
3568/3568 [==============================] - 13s 4ms/step - loss: 0.0060 - mae: 0.0060 - mse: 2.8286e-04 - val_loss: 0.0056 - val_mae: 0.0056 - val_mse: 3.0605e-04
Epoch 3/10
3568/3568 [==============================] - 15s 4ms/step - loss: 0.0057 - mae: 0.0057 - mse: 2.7686e-04 - val_loss: 0.0055 - val_mae: 0.0055 - val_mse: 2.9563e-04
Epoch 4/10
3568/3568 [==============================] - 13s 4ms/step - loss: 0.0056 - mae: 0.0056 - mse: 2.7382e-04 - val_loss: 0.0053 - val_mae: 0.0053 - val_mse: 2.9897e-04
Epoch 5/10
3568/3568 [==============================] - 13s 4ms/step - loss: 0.0055 - mae: 0.0055 - mse: 2.7098e-04 - val_loss: 0.0051 - val_mae: 0.0051 - val_mse: 3.0429e-04
Epoch 6/10
3568/3568 [==============================] - 13s 4ms/step - loss: 0.0054 - mae: 0.0054 - mse: 2.6773e-04 - val_los